# Logistic 回归

In [ ]:
import gzip
from time import time
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["font.sans-serif"] = "SimHei"
plt.rcParams["axes.unicode_minus"] = False
plt.tight_layout()

使用梯度下降法实现 Logistic 回归。

In [ ]:
class LogisticRegression:
    def __init__(self, learning_rate=0.1, max_iter=500):
        self._learning_rate = learning_rate
        self._max_iter = max_iter
        self._losses = []

    def _logistic(self, z):
        return 1 / (1 + np.exp(-z))

    def _loss(self, h, y):
        return -np.mean(y * np.log(h) + (1 - y) * np.log(1 - h))

    @property
    def _record_loss_interval(self):
        return max(1, self._max_iter // 10)

    def fit(self, X, y):
        m, n = X.shape

        self._theta = np.random.randn(m, 1) * 0.01

        for iteration in range(self._max_iter):
            z = np.dot(self._theta.T, X)
            h = self._logistic(z)
            gradient = np.dot(X, (h - y).T) / n
            self._theta -= self._learning_rate * gradient

            if (iteration + 1) % self._record_loss_interval == 0:
                self._losses.append(self._loss(h, y))

    def predict(self, X):
        z = np.dot(self._theta.T, X)
        h = self._logistic(z)
        return int(h > 0.5)

    def draw_loss(self):
        plt.plot(
            np.arange(1, len(self._losses) + 1) * self._record_loss_interval,
            self._losses,
        )
        plt.xlabel("迭代次数")
        plt.ylabel("损失函数值")
        plt.title("损失函数曲线")
        plt.show()

加载数据集的 helper 函数。

In [ ]:
def load_dataset(kind):
    labels_path = f"{kind}-labels-idx1-ubyte.gz"
    with gzip.open(labels_path, "rb") as fr:
        labels = np.frombuffer(fr.read(), dtype=np.uint8, offset=8)

    images_path = f"{kind}-images-idx3-ubyte.gz"
    with gzip.open(images_path, "rb") as fr:
        images = np.frombuffer(fr.read(), dtype=np.uint8, offset=16) / 255
        images = images.reshape(labels.shape[0], 784).T
        images = np.vstack((np.ones(labels.shape[0]), images))

    filter_indices = np.where((labels == 0) | (labels == 1))[0]
    labels = labels[filter_indices]
    images = images[:, filter_indices]

    shuffle_indices = np.random.permutation(labels.shape[0])
    labels = labels[shuffle_indices]
    images = images[:, shuffle_indices]

    return images, labels

训练模型，绘制损失函数曲线。

In [ ]:
train_images, train_labels = load_dataset("train")
test_images, test_labels = load_dataset("t10k")

elapsed_time = -time()
model = LogisticRegression()
model.fit(train_images, train_labels)
elapsed_time += time()

print(f"Optimization took {elapsed_time:.2f} seconds.")
model.draw_loss()

在训练集和测试集上分别测试准确率。

In [ ]:
train_predictions = [model.predict(image) for image in train_images.T]
train_accuracy = np.mean(train_predictions == train_labels) * 100
print(f"Training accuracy: {train_accuracy:.2f}%")

test_predictions = [model.predict(image) for image in test_images.T]
test_accuracy = np.mean(test_predictions == test_labels) * 100
print(f"Test accuracy: {test_accuracy:.2f}%")